In [82]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
#read csv files in data
train = pd.read_csv('/content/drive/MyDrive/data/train.csv')
test= pd.read_csv('/content/drive/MyDrive/data/test.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [83]:
#save row id in a dataframe
row_id = test['row ID']
#drop it from test now
test = test.drop(columns=['row ID'])

In [73]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [84]:
#convert data types where coluhmns have values yes and no to 1 and 0
train = train.replace({'yes': 1, 'no': 0})
test = test.replace({'yes': 1, 'no': 0})

#Create an instance of LabelEncoder
lbl = LabelEncoder()
#do label encoding for prodcut type as well
lbl.fit(list(train['product_type'].values) + list(test['product_type'].values))
train['product_type'] = lbl.transform(list(train['product_type'].values))
test['product_type'] = lbl.transform(list(test['product_type'].values))
train.drop(columns=['sub_area'], inplace=True)
test.drop(columns=['sub_area'], inplace=True)


In [92]:
categorical_features = train.select_dtypes(include=['object']).columns
categorical_features

Index([], dtype='object')

In [91]:
from sklearn.preprocessing import LabelEncoder

# Assuming lbl is a LabelEncoder instance
lbl = LabelEncoder()

# Fit the label encoder on the 'ecology' column in the training data
lbl.fit(train['ecology'])

# Convert the 'ecology' column in the train dataset
train['ecology'] = lbl.transform(list(train['ecology'].values))

# Convert the 'ecology' column in the test dataset using the same label encoder
test['ecology'] = lbl.transform(list(test['ecology'].values))


In [93]:
X = train.drop(columns=['price_doc'])
y = train[['price_doc']]

In [100]:
import statsmodels.api as sm

# Add a constant column to the feature matrix
XSM = sm.add_constant(X)

# Fit the ordinary least squares (OLS) model
model = sm.OLS(train['price_doc'], XSM).fit()

# Get the p-values of all columns
p_values = model.pvalues

# Sort the p-values in ascending order
sorted_p_values = p_values.sort_values()

# Show only the bottom 35 columns with the lowest p-values
bottom_35_p_values = sorted_p_values[:35]



In [122]:
# Get the absolute values of the coefficients as feature importance
feature_importance = abs(model.params)

# Sort the feature importance in descending order
sorted_feature_importance = feature_importance.sort_values(ascending=False)

# Show only the top 50 features with the highest importance
top_50_features = sorted_feature_importance[:50]

# Print the top 50 features
# print(top_50_features)


In [126]:
top_50_features.drop('const', inplace=True)


In [127]:
# Use the bottom 35 features above to train the model

# Select the bottom 35 variables from the dataset
X_new = X[list(top_50_features.index)]
# Do the same for the test dataset as well
test_new = test[list(top_50_features.index)]

In [128]:
from sklearn.preprocessing import MinMaxScaler

#Create a scaler object
scaler = MinMaxScaler()

#Fit the scaler and transform X_new
X_new = scaler.fit_transform(X)
test_new = scaler.fit_transform(test)

In [102]:
# Use the bottom 35 features above to train the model

# Select the bottom 35 variables from the dataset
X_new = X[list(bottom_35_p_values.index)]
# Do the same for the test dataset as well
test_new = test[list(bottom_35_p_values.index)]

In [52]:
numerical_features = X.select_dtypes(include=['float64', 'int64']).columns
categorical_features = X.select_dtypes(include=['object']).columns
#Create transformers for numerical and categorical features
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('ordinal', OrdinalEncoder())
])

# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Preprocess the training data
X_train_preprocessed = preprocessor.fit_transform(X)
X_test_preprocessed = preprocessor.transform(test)

In [55]:
X_train_preprocessed.shape

(181507, 271)

In [ ]:
%pip install catboost
%pip install xgboost
%pip install lightgbm
from catboost import CatBoostClassifier
import xgboost as xgb
import lightgbm as lgb

In [56]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn import tree
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [129]:
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=42)

In [135]:
# Create LGBMRegressor
lgb_model = lgb.LGBMRegressor(n_estimators=700,  # Number of boosting iterations
                              learning_rate=0.01,  # Step size shrinkage used in update to prevent overfitting
                              max_depth=20,  # Maximum depth of the trees
                              objective='regression',  # Loss function for regression
                              random_state=2, num_leaves=15)  # Random seed for reproducibility


# Fit the model on the training data
lgb_model.fit(X_train, y_train)
# Make predictions on the test set
predictions = lgb_model.predict(X_test, num_iteration=lgb_model.best_iteration_)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
print("Root Mean Squared Error: %.2f" % rmse)
#  12712668.5

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.642781 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 65311
[LightGBM] [Info] Number of data points in the train set: 145205, number of used features: 270
[LightGBM] [Info] Start training from score 14733110.056808
Root Mean Squared Error: 12712668.57


In [137]:
price_doc = lgb_model.predict(test_new)


In [138]:
#Create a DataFrame for the predictions
predictions_df = pd.DataFrame(price_doc, columns=['price_doc'])

#Add the record ID from the test data to the predictions DataFrame
predictions_df.insert(0, 'row ID', row_id)

#Save the predictions to a CSV file
predictions_df.to_csv('download1me.csv', index=False)